In [1]:
import pandas as pd
import numpy as np

Thanks to Kriti Mahajan. This table was developed with study type features extracted from the meta data.
Code found here: https://colab.research.google.com/drive/1lay-yWJ5ohN94lQLHYIxtWMuDWtCRGy0#scrollTo=N-IhAeEkcxDe

Strategy:
1. Combine newest CoY dataset, Kaggle dataset, PubMed dataset and other annotation group's dataset all into one annotation table.
2. Feature engineer via revising ngram list/reverse engineer
3. If desperate, get synthetic labels via synthetic augmentation of text data (trade out word for synonym).

### Trying direct merge with mf and kaggle label set via sha.

In [151]:
mf = pd.read_csv('metadata_features-2.csv')
mf

,Unnamed: 0,paper_id,sha,title,abstract,publish_time,authors,url,Statistical Methods,statistical_method_details,Study Type,study_type_details,results,coronavirus,fatality,sample_size,paper_focus,variables_controlled_for
0,0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,Sequence requirements for RNA strand transfer ...,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",http://europepmc.org/articles/pmc125340?pdf=re...,[],NaN,['mutagenesis'],Here we present results of a comprehensive co-...,Here we present results of a comprehensive co-...,NaN,NaN,NaN,NaN,NaN
1,1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,"Crystal structure of murine sCEACAM1a[1,4]: a ...",CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu...",http://europepmc.org/articles/pmc125375?pdf=re...,[],NaN,[],NaN,Here we report the crystal structure of solubl...,NaN,NaN,NaN,NaN,NaN
2,2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,Synthesis of a novel hepatitis C virus protein...,Hepatitis C virus (HCV) is an important human ...,2001-07-16,"Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; ...",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,[],NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,Structure of coronavirus main proteinase revea...,The key enzyme in coronavirus polyprotein proc...,2002-07-01,"Anand, Kanchan; Palm, Gottfried J.; Mesters, J...",http://europepmc.org/articles/pmc126080?pdf=re...,['model'],Molecular modelling and mutagenesis data impli...,['mutagenesis'],Molecular modelling and mutagenesis data impli...,The study illustrates that RNA viruses have ev...,NaN,NaN,NaN,NaN,NaN
4,4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,Discontinuous and non-discontinuous subgenomic...,"Arteri-, corona-, toro- and roniviruses are ev...",2002-12-01,"van Vliet, A.L.W.; Smits, S.L.; Rottier, P.J.M...",http://europepmc.org/articles/pmc136939?pdf=re...,[],NaN,[],NaN,We show that BEV mRNAs 3–5 lack a leader seque...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59125,59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,Cutting-Edge Infectious Disease Diagnostics wi...,"Three recent Science articles (Chen et al., 20...",2018-06-13,"Chiu, Charles",https://doi.org/10.1016/j.chom.2018.05.016,[],NaN,['clinical'],"Three recent Science articles (Chen et al., 20...",NaN,NaN,NaN,NaN,NaN,NaN
59126,59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,Intranasal administration of oxytocin: Behavio...,Abstract The intranasal (IN-) administration o...,2013-09-30,"Veening, Jan G.; Olivier, Berend",https://doi.org/10.1016/j.neubiorev.2013.04.012,[],NaN,['review'],The present review explores our current knowle...,Only very recently in a microdialysis study in...,NaN,NaN,NaN,NaN,NaN
59127,59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,Infecciones virales graves en pacientes inmuno...,Resumen Los virus tienen un papel importante d...,2011-04-30,"Díaz, A.; Zaragoza, R.; Granada, R.; Salavert, M.",https://doi.org/10.1016/j.medin.2011.01.001,[],NaN,['review'],A review of the literature has been performed ...,NaN,NaN,"Furthermore, ventilator-associated pneumonia (...",NaN,NaN,NaN
59128,59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,"18 Omics, Bioinformatics, and Infectious Disea...",Bioinformatics and the ‘omics’ studies are rap...,2011-12-31,"Paszkiewicz, Konrad H.; Giezen, Mark van der",https://doi.org/10.1016/b978-0-12-384890-1.000...,"['sequencing', 'methods']","In addition, we provide an overview of methods...",['bioinformatics'],Bioinformatics and the ‘omics’ studies are rap...,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
list(mf)

['Unnamed: 0',
 'paper_id',
 'sha',
 'title',
 'abstract',
 'publish_time',
 'authors',
 'url',
 'Statistical Methods',
 'statistical_method_details',
 'Study Type',
 'study_type_details',
 'results',
 'coronavirus',
 'fatality',
 'sample_size',
 'paper_focus',
 'variables_controlled_for']

In [153]:
features = mf.drop(columns = ['Unnamed: 0',
 'title',
 'abstract',
 'publish_time',
 'authors',
 'url',
 'Statistical Methods',
 'statistical_method_details',
 'study_type_details',
 'results',
 'coronavirus',
 'fatality',
 'sample_size',
 'paper_focus',
 'variables_controlled_for'])
features

,paper_id,sha,Study Type
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis']
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,[]
2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,[]
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis']
4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,[]
...,...,...,...
59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical']
59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review']
59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review']
59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics']


In [154]:
features['Study Type'].value_counts()

[]                                                                           37266
['clinical']                                                                  5782
['review']                                                                    3594
['genetic']                                                                   3089
['review', 'clinical']                                                        1347
                                                                             ...  
['retrospective study', 'retrospectively analyzed']                              1
['retrospective study', 'histories']                                             1
['review', 'retrospective study', 'retrospectively analyzed', 'clinical']        1
['review', 'single center study', 'clinical']                                    1
['meta-analysis', 'clinical', 'bioinformatics']                                  1
Name: Study Type, Length: 393, dtype: int64

In [155]:
to_drop = ['[]']
features = features[~features['Study Type'].isin(to_drop)]
features

,paper_id,sha,Study Type
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,['mutagenesis']
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,['mutagenesis']
6,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,"['genetic', 'histories']"
7,0m32ecnu,23bc55d6f63fab18b02004483888db2b6a0bfa48,['genetic']
10,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,['genetic']
...,...,...,...
59125,pwq5gir9,20f185b11fa23da8938efb5c839c6c078c66f1e9,['clinical']
59126,3racv2qe,e84532333db8f219f20ae0a8fe18130c0d010b04,['review']
59127,jngto9kv,31105078a2953217223699d09c6a80d0f5edfdf6,['review']
59128,td0776wj,b1f977d58a0eba25acdee5912ef71c372b0632de,['bioinformatics']


In [156]:
features['Study Type'].value_counts()

['clinical']                                                        5782
['review']                                                          3594
['genetic']                                                         3089
['review', 'clinical']                                              1347
['experimental']                                                    1241
                                                                    ... 
['review', 'retrospective cohort', 'case series', 'clinical']          1
['review', 'retrospective study', 'clinical', 'laboratory data']       1
['experimental', 'meta-analysis']                                      1
['time series', 'time-series', 'clinical']                             1
['mutagenesis', 'review', 'life cycle']                                1
Name: Study Type, Length: 392, dtype: int64

In [157]:
kaggle_labels = pd.read_csv('design-2.csv')

In [158]:
kg_merge = features.merge(kaggle_labels, how='inner', left_on='sha', right_on='id')
kg_merge

,paper_id,sha,Study Type,label,id
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e
...,...,...,...,...,...
493,dycx8i7h,2d0d272a793b99ef896547024c51f486e9552e5a,['genetic'],0,2d0d272a793b99ef896547024c51f486e9552e5a
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],2,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",7,95d9bac431797e2cf8b7294a648ceec3bd50c84b
496,i2vg8zqc,e9dfbc4084497e74dcaf6bd9c1dbde2b0fb70802,"['experimental', 'clinical']",0,e9dfbc4084497e74dcaf6bd9c1dbde2b0fb70802


Complete kg_merge annotation set by:
1. mapping labels to cw_labels
2. performing one-hot-encoding to output multi-label dataset

In [161]:
cw_labels = pd.read_csv('CoY_newlist.csv')

In [162]:
list(cw_labels)

['Assignee',
 'cord_uid',
 'sha',
 'title',
 'url',
 'Computational ',
 'Experimental-in vitro',
 'Experimental-in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional.1',
 'Clinical-observational',
 'Clinical-observational.1',
 'Systematic review and/or meta-analysis ',
 'Other',
 'General Notes']

In [163]:
cw_labels = cw_labels.drop(columns = ['Other', 'General Notes'])

In [164]:
#Rename/Clean-up labels
cw_labels = cw_labels.rename(columns={'Computational ': 'Computational', 'Experimental-in vitro' : 'Experimental - in vitro', 'Experimental-in vivo' : 'Experimental - in vivo', 'Clinical-interventional.1' : 'Clinical-interventional', 'Clinical-observational.1' : 'Clinical-observational', 'Systematic review and/or meta-analysis ' : 'Systematic review and/or meta-analysis'})

In [165]:
list(cw_labels)

['Assignee',
 'cord_uid',
 'sha',
 'title',
 'url',
 'Computational',
 'Experimental - in vitro',
 'Experimental - in vivo',
 'Systematic review and/or meta-analysis',
 'Clinical-interventional',
 'Clinical-interventional',
 'Clinical-observational',
 'Clinical-observational',
 'Systematic review and/or meta-analysis']

MAPPING SCHEME

Mapping Kaggle labels to CoronaWhy labels

0. Other -> Drop
1. Systematic review -> Systematic review and/or meta-analysis
2. Randomized control trial -> Clinical-interventional
3. Non-randomized trial -> Clinical - interventional
4. Prospective observational -> Clinical -observational
5. Time-to-event analysis -> Clinical-observational
6. Retrospective observational -> Clinical-observational
7. Cross-sectional -> Clinical-observational
8. Case series -> Clinical-observational
9. Modeling -> Computational

In [167]:
kg_merge.explode('label')
kg_merge['label'].unique()

array([1, 8, 5, 7, 4, 2, 9, 6, 0, 3])

In [168]:
dic_map={
         1:'Systematic review and/or meta-analysis',
         2:'Clinical-interventional',
         3:'Clinical-interventional',
         4:'Clinical-observational',
         5:'Clinical-observational',
         6:'Clinical-observational',
         7:'Clinical-observational',
         8:'Clinical-observational',
         9:'Computational',
         0:'Other'
        }

In [169]:
kg_merge['mapped'] = kg_merge.apply(lambda x: dic_map[x.label], axis = 1) 
kg_merge.head()

,paper_id,sha,Study Type,label,id,mapped
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b,Systematic review and/or meta-analysis
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,Clinical-observational
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,Clinical-observational
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541,Clinical-observational
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e,Clinical-observational


In [171]:
#Drop 0. Other label
kg_merge = kg_merge[kg_merge.label != 0]
kg_merge

,paper_id,sha,Study Type,label,id,mapped
0,47ema2dq,6a3fa8ed278df0d05c5e009521de11c72308f60b,['review'],1,6a3fa8ed278df0d05c5e009521de11c72308f60b,Systematic review and/or meta-analysis
1,9trzqzep,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,"['questionnaire', 'cross-sectional']",8,eb4b8f43f68db5e493984b7e4e56d9f804d7e885,Clinical-observational
2,hnu8gw6w,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,"['time series', 'clinical']",5,2afcf2ab2330cb56ec493f82d87fa0c5b5f076e9,Clinical-observational
3,5ulk3euw,9f7ed2037968c63fa97eb6fc7c35e9738a654541,['cross-sectional'],7,9f7ed2037968c63fa97eb6fc7c35e9738a654541,Clinical-observational
4,loowpch7,de707f97b534ca340482600ed1f008b20d7b1b8e,"['prospective cohort', 'prospective cohort']",4,de707f97b534ca340482600ed1f008b20d7b1b8e,Clinical-observational
...,...,...,...,...,...,...
490,l4649du3,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,['clinical'],2,bb3bf36ce2af876dcbf80b6cf3598e4421c98b3e,Clinical-interventional
491,3vpoxfnv,eb720970f362e4416b73393f230c8606c837c6dc,['clinical'],9,eb720970f362e4416b73393f230c8606c837c6dc,Computational
494,p8vhi6d9,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,['experimental'],2,f778c6317cdc3e2c4aa7a8c7734fa13da10723eb,Clinical-interventional
495,a6i73v0o,95d9bac431797e2cf8b7294a648ceec3bd50c84b,"['questionnaire', 'survey', 'cross-sectional']",7,95d9bac431797e2cf8b7294a648ceec3bd50c84b,Clinical-observational


100 less annotations is a huge drop. 

### One-Hot Encoding